In [2]:
#Importar librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import FuncAnimation
from sympy import symbols, Matrix, pi,deg, cos, sin,atan, simplify, eye, solve, latex, atan2, pprint, init_printing, Derivative, sqrt
from sympy.physics.mechanics import ReferenceFrame, dynamicsymbols, init_vprinting, Point
from sklearn.metrics import mean_squared_error
from scipy.integrate import cumulative_trapezoid

-----------------------------------------------------------------PUNTO 1---------------------------------------------------------

In [5]:
#variables
theta = dynamicsymbols("theta") #creamos al angulo dinamico, con respecto al tiempo
t = symbols("t") #creamos el tiempo

l = symbols("l") #mm,creamos a la longitud L simbolica

#marcos de referencia
N_fijo = ReferenceFrame("N_fijo")
A_movil = N_fijo.orientnew("A_movil", "Axis", (np.pi - theta, N_fijo.z)) #El angulo se mide desde el otro eje. Entonces en realidad el angulo es 180 - valor angulo


#Ahora bien, hasta aqui solo hemos llegado a A
#Necesitamos llegar a la punta de A, por lo que podemos crear otro marco de referencia
#Este nuevo sistema, por la friccion rota igual que el A pero en sentido contrario

B_movil = A_movil.orientnew("B_movil", "Axis", (-theta, A_movil.z)) #va en sentido contrario al otro

#Ahora si, ya podemos llegar a la punta:
radio = symbols("r") #mm
vector_pos = (l)*A_movil.x + radio*B_movil.x #que depende del sistema A y del sistema B

#Ahora bien, notamos que el vector de posicion que tenemos va a comportarse igual en terminos de aceleracion que el B, luego

velocidad_vector_pos = vector_pos.diff(t,N_fijo)
aceleracion_vector_pos = velocidad_vector_pos.diff(t, N_fijo)

aceleracion_vector_pos.magnitude().simplify()


4.0*sqrt(0.0625*l**2*Derivative(theta(t), t)**4 + 0.0625*l**2*Derivative(theta(t), (t, 2))**2 - 0.25*l*r*sin(theta(t))*Derivative(theta(t), t)**2*Derivative(theta(t), (t, 2)) + 0.5*l*r*cos(theta(t))*Derivative(theta(t), t)**4 + 0.25*l*r*cos(theta(t))*Derivative(theta(t), (t, 2))**2 + r**2*Derivative(theta(t), t)**4 + 0.25*r**2*Derivative(theta(t), (t, 2))**2)

In [ ]:
#Ahora para el problema en particular:
aceleracion_vector_pos.subs({
    theta: np.deg2rad(60),
    theta.diff(t): -4, 
    theta.diff(t,2):-8,
    l:0.080,
    radio:0.020}).magnitude().simplify()

1.62943180611531

-----------------------------------------------------------------PUNTO 2---------------------------------------------------------

In [10]:
#A. Describir la posicion de las barras con vectores
#Podemos crear un eje fijo en O:
#Ahora creamos al angulo y el tiempo
eje_fijo_O = ReferenceFrame("eje_fijo_O")
theta = dynamicsymbols("theta") #creamos al angulo dinamico, con respecto al tiempo
t = symbols("t") #creamos el tiempo
#Podemos tener

#Creamos la distancia entre deslizadores
x = symbols("x")
#Creamos la longitud de las barras
b = symbols("b")

#Creamos un marco de referencia movil para cada barra
eje_movil_C = eje_fijo_O.orientnew("eje_movil_C", "Axis", (theta, eje_fijo_O.z))   
eje_movil_D = eje_fijo_O.orientnew("eje_movil_D", "Axis", (np.pi -theta, eje_fijo_O.z)) #el angulo que mide C. El angulo de elevancion de C tambien es theta por el teorema del seno, solo que su angulo medido desde el eje Fijo es 180 - angulo
#Creamos los vectores posicion
vector_pos_O = b*eje_movil_C.x
vector_pos_C = b*eje_movil_D.x + x*eje_fijo_O.x #recordemos que el deslizador C esta desplazado X unidades con respecto al marco de referencia fijo

vector_pos_C

x*eje_fijo_O.x + b*eje_movil_D.x

In [11]:
#B
velocidad_C = vector_pos_C.diff(t, eje_fijo_O).simplify()
velocidad_C

- b*sin(theta(t) - 3.14159265358979)*Derivative(theta(t), t)*eje_fijo_O.x - b*cos(theta(t) - 3.14159265358979)*Derivative(theta(t), t)*eje_fijo_O.y

In [12]:
#C
aceleracion_C = velocidad_C.diff(t, eje_fijo_O).simplify()
aceleracion_C

b*(-sin(theta(t) - 3.14159265358979)*Derivative(theta(t), (t, 2)) - cos(theta(t) - 3.14159265358979)*Derivative(theta(t), t)**2)*eje_fijo_O.x + b*(sin(theta(t) - 3.14159265358979)*Derivative(theta(t), t)**2 - cos(theta(t) - 3.14159265358979)*Derivative(theta(t), (t, 2)))*eje_fijo_O.y

-----------------------------------------------------------------PUNTO 3---------------------------------------------------------

In [ ]:
#Crear el marco de referencia fijo y el angulo y el tiempo
fijo_3 = ReferenceFrame("fijo_3")
theta = dynamicsymbols("theta") #creamos al angulo dinamico, con respecto al tiempo
t = symbols("t") #creamos el tiempo
#Crear marco de referencia movil
movil_3 = fijo_3.orientnew("movil_3", "Axis", (theta, fijo_3.z))
#Crear vector posicion hacia A
b = symbols("b")
distancia_r = b/cos(theta)  #r cambia con respecto al angulo
vector_pos_3 = distancia_r*movil_3.x
#Crear vector desde el origen hasta la base de todo el objeto derecho
vector_base = b*fijo_3.x

#Crear vector de la altura y, el cual es la 
#Podemos hacer una suma de todos: 
#Vector de OA + Vector A al piso + Vector del piso a O = 0
#Luego Vector A al piso (y) = -vector OA (vector_pos_3) - vector del piso a O (vector_base)
vector_altura_y = vector_pos_3 - vector_base

#Derivar para obtener la velocidad
vector_velocidad_y = vector_altura_y.diff(t, fijo_3)
#Derivar otra vez para obtener aceleracion
vector_aceleracion_y = vector_velocidad_y.diff(t, fijo_3)

vector_aceleracion_y.simplify()

b*(2*sin(theta(t))*Derivative(theta(t), t)**2 + cos(theta(t))*Derivative(theta(t), (t, 2)))/cos(theta(t))**3*fijo_3.y

-----------------------------------------------------------------PUNTO 4---------------------------------------------------------

In [ ]:
#Definir el volumen
hc = symbols("hc")
r_inicial = symbols("r_0")
diferencia_r = symbols("delta_r")
volumen_inicial = (np.pi*(r_inicial**2)*hc)
volumen_final = symbols("v_final")

cambio_tornillo = 1.25 #mm, por vuelta

#Definir la cantidad de vueltas
num_vueltas = diferencia_r*(1/cambio_tornillo)

#Hallar el delta r
ecuacion = Eq(diferencia_r**2 + diferencia_r*r_inicial + volumen_final- volumen_inicial, 0)
solucion = solve(ecuacion, diferencia_r)
print(solucion)

In [ ]:
# Variables simbólicas
delta_r = symbols("delta_r", real=True)
r0 = 0        # Radio inicial (mm)
h = 150        # Altura del cilindro (mm)
v_inicial = 0 # Volumen inicial (mm³)
cambio_tornillo = 1.25  # mm por vuelta

# Rango de volúmenes
volumenes = np.linspace(0, 720000, 100)

vueltas = []

for v_final in volumenes:
    # Ecuación completa: π*h*(2*r0*delta_r + delta_r**2) = v_final - v_inicial
    eq = Eq(np.pi*h*(2*r0*delta_r + delta_r**2), v_final - v_inicial)
    soluciones = solve(eq, delta_r)
    
    # Filtrar solo soluciones reales y positivas
    soluciones_reales = [s.evalf() for s in soluciones if s.is_real and s.evalf() >= 0]
    
    if soluciones_reales:
        delta_r_val = min(soluciones_reales)  # Tomamos la menor positiva (normalmente la válida)
        vueltas.append(delta_r_val / cambio_tornillo)
    else:
        vueltas.append(np.nan)  # Si no hay solución real

# Graficar
plt.plot(vueltas, volumenes, label='Volumen vs Vueltas')
plt.ylabel("Volumen final (mm³)")
plt.xlabel("Número de vueltas")
plt.title("Relación Volumen - Vueltas del Tornillo")
plt.grid(True)
plt.legend()
plt.show()